# Module 5: Networking

## Exercise 1: Working with Networking

In this exercise, you explore the networking features of a Kubernetes cluster.

**Objectives**

This exercise focuses on enabling you to do the following:
  - Explore the network components
  - Understand pod networking
  - Investigate service networking
  - Review the DNS configuration
  - Create front-end, back-end, and admin environments
  - Explore regulating network traffic by using a network policy


## Exercise Equipment

In this exercise, you use the following systems.

| System                  | Host Name   | IP Addresses   | User Name (case sensitive) | Password  |
|-------------------------|-------------|----------------|----------------------------|-----------|
| Linux Mint 20           | jumphost    | 192.168.0.5    | user                       | Netapp1!  |
| Kubernetes Control Plane| kubmas1-1   | 192.168.0.61   | root                       | Netapp1!  |
| Kubernetes Worker 1     | kubwor1-1   | 192.168.0.62   | root                       | Netapp1!  |
| Kubernetes Worker 2     | kubwor1-2   | 192.168.0.63   | root                       | Netapp1!  |
| Kubernetes Worker 3     | kubwor1-3   | 192.168.0.64   | root                       | Netapp1!  |

---
---

##### Task 1: Explore the Network Components

In this task, you review the network components of your Kubernetes cluster.


---

List the available nodes in your cluster:


In [ ]:
kubectl get nodes


---

Answer the following question:

What is the IP address that is assigned to the control plane?


In [ ]:

kubectl get nodes -o wide


---

Create a Secure Shell (SSH) connection to the kubmas1-1 node.

**OR**

You can use the cells below to remotely execute code on kubmas1-1

---

Review the network interfaces on the control plane:


In [ ]:

ssh root@kubmas1-1 ifconfig -a


---

Identify a network interface that is configured for cluster connectivity on the control plane:


In [ ]:
ssh root@kubmas1-1 ip link


---

Answer the following question:

What is the MAC address that is assigned to the control plane?



---

Answer the following question:

What is the address that is assigned to node kubwor1-1?


In [ ]:
kubectl get nodes -o wide


---

Answer the following question:

What is the MAC address that is assigned to kubwor1-1?



In [ ]:
ssh root@kubmas1-1 arp Kubwor1-1.demo.netapp.com


---

Answer the following question:

What is the address that is assigned to the node kubwor1-2?


In [ ]:
kubectl get nodes -o wide


---


Answer the following question:

What is the MAC address that is assigned to the kubwor1-2?



In [ ]:
ssh root@kubmas1-1 arp Kubwor1-2.demo.netapp.com


---

Answer the following question:

What is the weave interface or bridge on the control plane, and what is its state?


In [ ]:
ssh root@kubmas1-1 ip link


In [ ]:
ssh root@kubmas1-1 ip link |grep weave


---

Answer the following question:

What is the state of the weave interface?



In [ ]:
ssh root@kubmas1-1 ip link show weave


---

Answer the following question:

What is the default gateway?



In [ ]:
ssh root@kubmas1-1 ip route show default


---

Answer the following question:

On the control plane, on which port does the Kubernetes scheduler listen?



In [ ]:
ssh root@kubmas1-1 netstat -nplt


In [ ]:
ssh root@kubmas1-1 netstat -nplt |grep kube-scheduler


---

Notice that etcd listens on three different ports across four addresses.


In [ ]:
ssh root@kubmas1-1 netstat -nplt |grep etcd


---

Answer the following question:

Which ectd port has more client connections?



In [ ]:
ssh root@kubmas1-1 netstat -anp | grep etcd


---
---

##### Task 2: Understand Pod Networking

In this task, you learn how to configure networking for the pods.


---

Create an SSH connection to one of the worker nodes in your Kubernetes cluster.

*OR*

You can use the cells below to remotely execute code on kubwor1-1

---

Run netstat -nplt on a node and compare the services that are running on a worker to the
services that are running on the control plane.


In [ ]:
diff <(ssh root@kubwor1-1 netstat -nplt |sort) <(ssh root@kubmas1-1 netstat -nplt|sort)

---

Investigate the kubelet service, and identify the network plug-in being used in Container
Network Interface (CNI)-compliant:



In [ ]:
ssh root@kubwor1-1 ps -aux | grep network


---

Identify a list of possible CNI plug-ins:


ls /opt/cni/bin


In [ ]:
ssh root@kubwor1-1 ls /opt/cni/bin


In [ ]:
diff <(ssh root@kubwor1-1 ls /opt/cni/bin) <(ssh root@kubmas1-1 ls /opt/cni/bin)

---

Compare the running process with network connection to the list of possible CNI plug-ins, and
identify the current running CNI plug-in:



netstat -nplt


In [ ]:
ssh root@kubwor1-1 netstat -nplt | sort -k6


In [ ]:
ssh root@kubwor1-1 netstat -nplt | awk '{print $7}' | sort  


---

Review how the Weave network is configured:


In [ ]:
ssh root@kubwor1-1 cat /etc/cni/net.d/10-weave.conflist


---

The /etc/cni/net.d directory is the default location for a CNI plug-in. 

If the location is different, you need to configure the Kubelet with the --cni-conf-dir parameter.


---

Identify how many and which nodes are Weave agents that are deployed in the cluster:


In [ ]:
kubectl get pods -n kube-system -o wide | grep weave 



---

The `kubectl get pods -n kube-system` command fails if you issue it on the Kubernetes nodes. 

Do you know why? 

You could resolve this or issue the command on a node on which the kubeconfig has been properly configured.


---

For each node in the cluster, identify the name of the Weave (bridge) network:


for node in (kubmas1-1 kubwork1-1 kubwork1-2 kubwork1-3) ; do ssh  ip link


In [ ]:
for host in kubmas1-1 kubwor1-1 kubwor1-2 kubwor1-3; do
    echo "Checking $host..."
    ssh root@$host "ip link | grep weave"
    echo "========"
done

---

Identify the pod IP address range configured for your CNI plug-in on the control plane and
nodes:


In [ ]:
for host in kubmas1-1 kubwor1-1 kubwor1-2 kubwor1-3; do
    echo "Checking $host..."
    ssh root@$host "ip addr show weave "
    echo "========"
done

---

Answer the following question:

Can you identify the default gateway that is configured on the pods that are scheduled on your
node?

ip route


In [ ]:
for host in kubmas1-1 kubwor1-1 kubwor1-2 kubwor1-3; do
    echo "Checking $host..."
    ssh root@$host "ip route show default "
    echo "========"
done

---

##### Task 3: Investigate Service Networking  

In this task, you investigate how networking functions with services in a Kubernetes cluster.  


---

A service is an abstraction for pods that provides a stable virtual IP (VIP) address.

Although pods and their IP addresses might come and go, a service enables clients
to connect reliably to the containers in the pods by using VIP addresses. 

The virtual in VIP means that the address is not an actual IP address connected to a network interface. 

The purpose of a VIP address is purely to forward traffic to one or more pods. 

The kube-proxy process keeps the mapping between the VIP and the pods up to date. 

The kube-proxy process runs on every node that queries the API server and learns about new services  in the cluster.


---

Create an alpine pod, a pod that is supervised by a replication controller, and a service to
access the replication controller pod:

<details> <summary> exercise5task3.yaml </summary> 

[exercise5task3.yaml](./exercise5task3.yaml)  
```yaml
apiVersion: v1
kind: ReplicationController
metadata:
  name: rcsise
spec:
  replicas: 1
  selector:
    app: sise
  template:
    metadata:
      name: name
      labels:
        app: sise
    spec:
      containers:
      - name: sise
        image: mhausenblas/simpleservice:0.5.0
        imagePullPolicy: IfNotPresent
        ports:
        - containerPort: 9876
---
apiVersion: v1
kind: Service
metadata:
  name: simpleservice
spec:
  ports:
    - port: 80
      targetPort: 9876
  selector:
    app: sise
---
apiVersion: v1
kind: Pod
metadata:
  name: alpine
  namespace: default
spec:
  containers:
  - image: alpine:3.19.1
    imagePullPolicy: IfNotPresent
    command:
      - /bin/sh
      - "-c"
      - "sleep 60m"
    name: alpine
    resources:
      requests:
        memory: "64Mi"
        cpu: "250m"
      limits:
        memory: "128Mi"
        cpu: "500m"
  restartPolicy: Always




In [ ]:
kubectl apply -f exercise5task3.yaml



---

Now get the supervised pod running:


In [ ]:
kubectl get pods -l app=sise


In [ ]:
pod=$(kubectl get pods -l app=sise |grep sise |awk '{print $1}')


In [ ]:

kubectl describe pod $pod


---

Access the alpine pod:


In [ ]:
kubectl exec alpine -it -- /bin/sh


---

Add curl to the alpine image:

`# apk --no-cache add curl`


---

You can access the pod directly from within the cluster by using the describe
operation to get the rcsise-XXX pod’s assigned IP address:

`# curl [ip-address-of-pod]:9876/info`

Sample output:
```json
{"host": "10.36.0.1:9876", "version": "0.5.0", "from": "10.42.0.1"}
```

In [ ]:
kubectl exec alpine -it -- /sbin/apk --no-cache add curl

In [ ]:
echo $IP ;kubectl exec alpine -it -- curl "$IP":9876/info

However, this process is not recommended because, as mentioned previously, a pod’s IP address might change. 

To avoid problems with IP addresses that change, you create the simple service:



In [ ]:
kubectl get svc


---

Record the IP address of the simple service:


In [ ]:
kubectl describe svc simpleservice


In [ ]:
SIP=$(kubectl describe svc simpleservice|grep -m1 ^IP:|awk '{print $2}');echo $SIP


---

From within the cluster, you can access your service by using the cluster IP in the command
from the previous step:

`# curl [ip-address-of-service]/info`

Sample output:
```json
{"host": "10.99.236.237", "version": "0.5.0", "from": "10.42.0.1"}
```

In this example, the service address 10.99.236.237 forwards traffic to the pod by using
iptables.

In [ ]:
echo $SIP ;kubectl exec alpine -it -- curl "$SIP"/info

---

To view the iptables for your simple service from Worker 1’s Linux SSH session (with root
access), use the following command:


In [ ]:
ssh root@kubwor1-1 iptables-save | grep simpleservice


---

Add a second pod by scaling up the replica:


In [ ]:
kubectl scale --replicas=2 rc/rcsise


---

Wait about 1 minute and then check the iptables again to display the new rules from Worker
1’s Linux SSH session:


In [ ]:
ssh root@kubwor1-1 iptables-save | grep simpleservice


---

Remove all the resources that you used:


In [ ]:
kubectl delete svc simpleservice
kubectl delete rc rcsise
kubectl delete pod alpine


##### Task 4: Review the DNS Configuration

In this task, you learn how DNS is configured in your cluster.


---

Identify which DNS system is deployed in your cluster:


In [ ]:
kubectl -n kube-system get pods


---

Answer the following question: 

How many pods are used for DNS?


In [ ]:
kubectl -n kube-system get pods|grep coredns 


---

Answer the following question: 

What service is used for DNS?


In [ ]:
kubectl -n kube-system get service


---

Answer the following question: 

What is the IP address for the DNS service?


---

Answer the following question: 

How does the file pass into the CoreDNS pod?


In [ ]:
kubectl -n kube-system get configmap


---

Answer the following question: 

What is the root domain?


In [ ]:
kubectl -n kube-system describe configmap coredns


---

Answer the following question: 

What is the fully qualified domain name (FQDN) for the DNS
service?


---

Create the exercise5task4.yaml pod.

<details> <summary>exercise5task4.yaml  </summary>

[exercise5task4.yaml](./exercise5task4.yaml)  

```yaml
apiVersion: v1
kind: Pod
metadata:
  name: alpine
  namespace: default
spec:
  containers:
  - image: alpine:3.19.1
    imagePullPolicy: IfNotPresent
    command:
      - /bin/sh
      - "-c"
      - "sleep 60m"
    name: alpine
    resources:
      requests:
        memory: "64Mi"
        cpu: "250m"
      limits:
        memory: "128Mi"
        cpu: "500m"
  restartPolicy: Always


In [ ]:
kubectl create -f exercise5task4.yaml

In [ ]:
kubectl get pods

---

Verify the configuration of the pod’s resolv.conf:


In [ ]:
kubectl exec alpine -- cat /etc/resolv.conf


---

Delete the alpine pod.


In [ ]:
kubectl delete pod alpine

##### Task 5: Create Front-End, Back-End, and Admin Environments

In this task, you create a namespace, a pod, and a ClusterIP service for three different environments
named frontend, backend, and admin. 

Then, you verify that all three environments can communicate. 

You can copy and paste these commands from the [exercise5task5.txt](./exercise5task5.txt) file.


---

Create the front-end namespace and objects by using imperative commands:


In [ ]:
kubectl create ns frontend
kubectl label namespaces frontend role=frontend
kubectl run web --image=nginx:1.25.4 --labels=app=web --port 80 -n frontend
kubectl expose pod web --type=ClusterIP --port=80 -n frontend


---

Create the back-end namespace and objects:


In [ ]:
kubectl create ns backend
kubectl label namespaces backend role=backend
kubectl run api --image=nginx:1.25.4 --labels=app=api --port 80 -n backend
kubectl expose pod api --type=ClusterIP --port=80 -n backend


---

Create the admin namespace and objects:


In [ ]:
kubectl create ns admin
kubectl label namespaces admin role=admin
kubectl run admin --image=nginx:1.25.4 --labels=app=admin --port 80 -n admin
kubectl expose pod admin --type=ClusterIP --port=80 -n admin


---

Verify that you can access all pods, regardless of the namespace:


In [ ]:
kubectl exec -it web -n frontend -- curl api.backend
kubectl exec -it web -n frontend -- curl admin.admin
kubectl exec -it admin -n admin -- curl web.frontend
kubectl exec -it admin -n admin -- curl api.backend
kubectl exec -it api -n backend -- curl web.frontend
kubectl exec -it api -n backend -- curl admin.admin


---
---

##### Task 6: Explore Regulating Network Traffic by Using a Network Policy

You now configure several network policies to restrict network traffic.


---

Create a network policy to deny all traffic to the front-end, back-end, and admin namespaces.

You can find the solution in [exercise5task6-1.yaml](./Solutions/exercise5task6-1.yaml) in the Solutions subfolder.


In [ ]:
kubectl create -f ./Solutions/exercise5task6-1.yaml

---

Verify that you created the network policies:


In [ ]:
kubectl get networkpolicy --all-namespaces


---

Verify that all pod connections time out when the pods try to communicate:


In [ ]:
kubectl exec -it admin -n admin -- curl api.backend
kubectl exec -it api -n backend -- curl web.frontend
kubectl exec -it web -n frontend -- curl admin.admin


Each of these commands might take up to 2.5 minutes to time out.


---

Remove the deny-all-traffic policies for the front-end and back-end namespaces.


In [ ]:
kubectl delete networkpolicy deny -n frontend


In [ ]:
kubectl delete networkpolicy deny -n backend


---

Create a network policy to enable only ingress to the web pod from the API pod, and then
deploy the policy.

You can find the solution in [exercise5task6-2.yaml](./Solutions/exercise5task6-2.yaml) in the Solutions subfolder.


In [ ]:
kubectl create -f Solutions/exercise5task6-2.yaml

---


Verify accessibility from the web pod to the back-end namespace:


In [ ]:
kubectl exec -it api -n backend -- curl web.frontend


---

Verify that all pod connections time out when the pods try to communicate with the admin pod:


In [ ]:
kubectl exec -it web -n frontend -- curl admin.admin


In [ ]:
kubectl exec -it api -n backend -- curl admin.admin


In [ ]:
kubectl exec -it api -n backend -- curl web.frontend


---

Verify the network policies that you created:


In [ ]:
kubectl get networkpolicy --all-namespaces


---

Remove all network policies.


In [ ]:
kubectl delete networkpolicy deny -n admin

In [ ]:
kubectl delete networkpolicy allow-backend-to-frontend -n backend

End of exercise
